In [1]:
from StockAnalysisFunctions2 import *
import matplotlib.pyplot as plt
import warnings
import csv
from scipy.optimize import minimize

# setup notebook
warnings.simplefilter("ignore")

plt.style.use('ggplot')
pd.set_option('precision', 3)
np.set_printoptions(precision=3)


# Make Plotly work Jupyter
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
# Use Plotly locally
cf.go_offline()

sfa = SFA()

## Read in Tickers from File and Collect Data

#### Enter Dates and File Names

In [2]:
# Set Data Pull Dates
START_DATE = dt.datetime.now() - dt.timedelta(weeks=2*52)
END_DATE = dt.datetime.now()

risk_free_rate = 0.02 # 10 year treasury yield

tick_fname = 'all_vaguard_admiral_and_sectors.csv'
desc_fname = 'all_vaguard_admiral_and_sectors_desc.csv'

In [3]:
# Get tickers for Vanguard Funds and ETF's
tickers = list(pd.read_csv(tick_fname).columns)

#### Set flag to load ticker descriptions from file and reset to get from yahoo

In [4]:
descriptions_from_file = True # set to read data from file, reset to collect form yahoo

if descriptions_from_file:
    descriptions = list(pd.read_csv(desc_fname))
else:
    descriptions = []
    for i,tick in enumerate(tickers):
        print(f'Collecting {tick} description {i+1} of {len(tickers)}')
        descriptions.append(sfa.get_ticker_description(tick))
    with open(desc_fname,'w') as f:
        write = csv.writer(f)
        write.writerow(descriptions)
    

In [5]:
stocks = {}
for i,tick in enumerate(tickers):
    print(f'Collecting Data For {tick} stock {i+1} of {len(tickers)}')
    stocks[tick] = {'data': sfa.get_stock_data(tick, START_DATE, END_DATE),
                   'description':descriptions[i]}

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

## Add Cumulative Return

In [6]:
for tick in tickers:
    stocks[tick]['data'] = sfa.add_daily_return(stocks[tick]['data'])
    stocks[tick]['data'] = sfa.add_cumulative_return(stocks[tick]['data'])

### Join Stocks in to DF on Cumulative Return and Close

In [7]:
def join_ticks_on_column(stocks_, column_):
    multi_df = pd.DataFrame()
    
    for key in stocks_.keys():
        multi_df[key] = stocks_[key]['data'][column_]
        
    return multi_df

In [8]:
mult_cum_df = join_ticks_on_column(stocks,'cumulative_return')
mult_close_df = join_ticks_on_column(stocks,'Close')

In [9]:
# Plot out cumulative returns for each stock since beginning of 2017
fig = px.line(mult_cum_df, x=mult_cum_df.index, y=mult_cum_df.columns,
             title = f'Stock Returns: {START_DATE.strftime("%d %b %Y")} to {END_DATE.strftime("%d %b %Y")}')
fig.update_xaxes(title="Date", rangeslider_visible=True)
fig.update_yaxes(title="Return")
fig.update_layout(height=500, width=900, 
                  showlegend=True)

fig.show()

In [10]:
returns = np.log(mult_close_df / mult_close_df.shift(1))
mean_ret = returns.mean() * 252 # 252 average trading days per year
mean_ret = pd.DataFrame(mean_ret.sort_values(ascending=False),columns=['annual_return'])
mean_ret['description'] = [stocks[x]['description'] for x in mean_ret.index]
mean_ret

,annual_return,description
VDE,0.596,Vanguard Energy Index Fund ETF Shares
VAW,0.426,Vanguard Materials Index Fund ETF Shares
VCR,0.426,Vanguard Consumer Discretionary Index Fund ETF...
VFH,0.401,Vanguard Financials Index Fund ETF Shares
VGT,0.394,Vanguard Information Technology Index Fund ETF...
VSMAX,0.387,Vanguard Small-Cap Index Fund Admiral Shares
VIS,0.378,Vanguard Industrials Index Fund ETF Shares
VIMAX,0.372,Vanguard Mid-Cap Index Fund Admiral Shares
VTSAX,0.351,Vanguard Total Stock Market Index Fund Admiral...
VFIAX,0.348,Vanguard 500 Index Fund Admiral Shares


### Returns Correlation

In [11]:
# We want great performing stocks that go up when others go down to smooth the returns over the year
pd.options.display.max_columns = 30
pd.options.display.max_rows = 30

rc = returns.corr()
rc

,VBTLX,VTABX,VTIAX,VTSAX,VTAPX,VBIAX,VFIAX,VIMAX,VSMAX,VDE,VAW,VIS,VPU,VHT,VFH,VDC,VCR,VGT,VOX,VNQ
VBTLX,1.000,0.619,-0.106,-0.066,0.218,0.057,-0.066,-0.064,-0.131,-0.226,-0.150,-0.161,0.036,0.005,-0.272,-0.013,-0.063,0.038,0.002,-0.018
VTABX,0.619,1.000,-0.086,-0.046,0.415,0.034,-0.047,-0.056,-0.088,-0.144,-0.099,-0.096,0.051,-0.015,-0.188,0.031,-0.060,0.014,0.013,-0.027
VTIAX,-0.106,-0.086,1.000,0.864,0.112,0.833,0.859,0.855,0.836,0.594,0.811,0.806,0.533,0.705,0.768,0.636,0.791,0.751,0.745,0.693
VTSAX,-0.066,-0.046,0.864,1.000,0.143,0.975,0.993,0.972,0.924,0.584,0.836,0.868,0.647,0.849,0.799,0.752,0.914,0.912,0.887,0.786
VTAPX,0.218,0.415,0.112,0.143,1.000,0.164,0.141,0.145,0.131,0.225,0.125,0.151,0.128,0.114,0.059,0.124,0.108,0.119,0.145,0.149
VBIAX,0.057,0.034,0.833,0.975,0.164,1.000,0.967,0.945,0.896,0.551,0.799,0.831,0.633,0.825,0.750,0.739,0.896,0.906,0.874,0.769
VFIAX,-0.066,-0.047,0.859,0.993,0.141,0.967,1.000,0.952,0.890,0.575,0.826,0.858,0.664,0.851,0.791,0.774,0.896,0.909,0.884,0.780
VIMAX,-0.064,-0.056,0.855,0.972,0.145,0.945,0.952,1.000,0.962,0.615,0.873,0.909,0.671,0.817,0.834,0.717,0.890,0.845,0.828,0.830
VSMAX,-0.131,-0.088,0.836,0.924,0.131,0.896,0.890,0.962,1.000,0.680,0.874,0.918,0.595,0.742,0.865,0.648,0.868,0.761,0.772,0.804
VDE,-0.226,-0.144,0.594,0.584,0.225,0.551,0.575,0.615,0.680,1.000,0.683,0.712,0.413,0.412,0.739,0.450,0.467,0.346,0.428,0.538


### Optimize Sharpe Ratio
#### Sharpe Ratio
People want to maximize returns while avoiding as much risk as possible. William Sharpe created the Sharpe Ratio to find the portfolio that provides the best return for the lowest amount of risk.

Sharpe Ratio

$\frac{\boldsymbol{r}^T\boldsymbol{x} - r_f}{\sqrt{\boldsymbol{x}^T\boldsymbol{Q}\boldsymbol{x}}}$
 

 $r_f$ = Risk Free Rate (annual 10-year bond yeild)

 $\boldsymbol{r}$ = Annual mean return rate of the stocks vector

 $\boldsymbol{Q}$ = Covariance matrix of stock returns
 
 $\boldsymbol{x}$ = Portfolio weights for stocks vector

The idea is to select stocks that maximize the ratio. A ration over 1.25 is considered good and close to 2 is exceptional.

In [12]:
'''Set Stocks To Evaluate'''
port_list = list(mean_ret.index)

In [13]:
'''Calculations for Sharpe Ratio'''

def get_volatility(X, cov_mat):
    return np.sqrt(252*np.matmul(np.matmul(X,cov_mat),np.transpose(X)))

def get_return(X, mean_annual_returns):
    return np.sum(X * mean_annual_returns)

def get_sharpe_ratio(X, cov_mat, mean_annual_returns, risk_free_rate_):
    vol = get_volatility(X,cov_mat)
    if vol > 0.0:
        return (get_return(X,mean_annual_returns) - risk_free_rate_)/vol
    else:
        return None

In [14]:
'''Run Optimization'''
num_stocks = len(port_list)

cov_matrix = returns[port_list].cov().values
mar_matrix = mean_ret.loc[port_list].annual_return.values

# turn in to minimization problem (we would like to maximize sharpe ratio)
obj_func = lambda X, cm, mar, rfr: -1.0*get_sharpe_ratio(X,cm,mar,rfr)

# equality constraint, weights must sum to 1 (result of eq constraint must be zero per minimize docs)
cons = ({'type':'eq','fun':lambda X: np.sum(X) - 1})

# bounds for weights is zero to one
X0 = np.array(num_stocks*[1.0/num_stocks]) # initial values uniform dist
bnds = tuple([(0.0,1.0) for x in X0])

res = minimize(obj_func, x0 = X0,
               args = (cov_matrix, mar_matrix, risk_free_rate),
               method = 'SLSQP',
               bounds = bnds,
               constraints = cons,
               tol = 1e-6,
               options = {'maxiter':1000000}
              )
print(res.message)
print(50*'=')
res

Optimization terminated successfully


     fun: -1.8654406004319648
     jac: array([-0.124, -0.123, -0.123,  0.248,  0.006,  0.343,  0.185,  0.125,
        0.103,  0.036,  0.195, -0.124,  0.376,  0.263, -0.123,  0.224,
        0.162, -0.124,  0.107,  0.179])
 message: 'Optimization terminated successfully'
    nfev: 296
     nit: 14
    njev: 14
  status: 0
 success: True
       x: array([1.471e-01, 3.373e-02, 3.027e-01, 0.000e+00, 0.000e+00, 1.145e-16,
       3.984e-17, 0.000e+00, 0.000e+00, 7.074e-18, 0.000e+00, 2.148e-01,
       3.756e-16, 0.000e+00, 8.237e-02, 3.307e-16, 0.000e+00, 2.193e-01,
       0.000e+00, 4.139e-17])

In [15]:
'''Display resulting portfolio'''
INVESTMENT = 300000.0 #dollars

port_df = mean_ret.loc[port_list].copy(deep=True).drop(columns='annual_return')
port_df['weights'] =  np.round(100.0*res.x,1)
port_df['dollars'] = np.round(INVESTMENT*res.x,-2)
s = port_df.sum(numeric_only=True)
s.name = 'totals'
port_df = port_df.append(s)

print(f'Final Sharpe Ratio: {get_sharpe_ratio(res.x, cov_matrix, mar_matrix, risk_free_rate):0.2f}')
print(f'Final Annual Volatility: {get_volatility(res.x, cov_matrix)*100:0.2f}%')
print(f'Final Annual Return: {get_return(res.x, mar_matrix)*100:0.2f}%')
print(f'Analysis Time Period: {END_DATE.strftime("%d %b %Y")} to {START_DATE.strftime("%d %b %Y")}')
print(f'Analysis Data Duration: {(END_DATE - START_DATE).total_seconds()/3600/24/365:0.2f} years')
display(port_df)

Final Sharpe Ratio: 1.87
Final Annual Volatility: 16.15%
Final Annual Return: 32.12%
Analysis Time Period: 19 Mar 2022 to 21 Mar 2020
Analysis Data Duration: 1.99 years


,description,weights,dollars
VDE,Vanguard Energy Index Fund ETF Shares,14.7,44100.0
VAW,Vanguard Materials Index Fund ETF Shares,3.4,10100.0
VCR,Vanguard Consumer Discretionary Index Fund ETF...,30.3,90800.0
VFH,Vanguard Financials Index Fund ETF Shares,0.0,0.0
VGT,Vanguard Information Technology Index Fund ETF...,0.0,0.0
VSMAX,Vanguard Small-Cap Index Fund Admiral Shares,0.0,0.0
VIS,Vanguard Industrials Index Fund ETF Shares,0.0,0.0
VIMAX,Vanguard Mid-Cap Index Fund Admiral Shares,0.0,0.0
VTSAX,Vanguard Total Stock Market Index Fund Admiral...,0.0,0.0
VFIAX,Vanguard 500 Index Fund Admiral Shares,0.0,0.0


### References
[Derek Banas' Github](https://github.com/derekbanas/Python4Finance/blob/main/Ultimate%20Portfolio.ipynb)

[Yet Another Math Programming Consultant](http://yetanothermathprogrammingconsultant.blogspot.com/2016/08/portfolio-optimization-maximize-sharpe.html)

[Optimization Methods in Finance](http://web.math.ku.dk/~rolf/CT_FinOpt.pdf)

[Kaggle Notebook, Lesson 6:Sharpe Ratio based Portfolio Optimization](https://www.kaggle.com/code/vijipai/lesson-6-sharpe-ratio-based-portfolio-optimization/notebook)